<a href="https://colab.research.google.com/github/dushanranasinghe/Google-Colab/blob/main/prediction_with_json_obj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install prophet

     |████████████████████████████████| 65 kB 2.6 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 214 kB 19.2 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639922 sha256=1bef690262e2ab916ec40bc1d4d1a7b4919b656cde7c12e6cacbda8ae5eb532e
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


In [1]:
def get_past_market_data(digital_currency, symbol, market, api_key):
  json_get = requests.get('https://www.alphavantage.co/query?function='+digital_currency+'&symbol='+symbol+'&market='+market+'&apikey='+api_key+'')

  frequency_type = get_data_frequency_type(digital_currency)

  data_attribute_name = 'Time Series (Digital Currency '+frequency_type+')'

  data_arr = json_get.json()[data_attribute_name]

  return data_arr

In [11]:
def get_data_frequency_type(digital_currency):
  return digital_currency.split("_")[2].capitalize() 

In [4]:
def organize_data_for_pandas(data_arr, market):
  new_data_arr = []
  DAY_HIGH = '2a. high ('+market+')'
  DAY_LOW = '3a. low ('+market+')'

  for obj in data_arr:
      json_obj = {}
      json_obj["ds"] = obj
      json_obj["y"] = (float(data_arr[obj][DAY_HIGH]) + float(data_arr[obj][DAY_LOW])) / 2
      new_data_arr.append(json_obj)

  strigify_arr = json.dumps(new_data_arr)

  return strigify_arr

In [5]:
def run_prophet(strigify_arr):
  df = pd.read_json(strigify_arr)
  df.head()

  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=365)
  future.tail()

  forecast = m.predict(future)
  forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

  fig1 = m.plot(forecast)

In [13]:
import pandas as pd
import json
import requests
from prophet import Prophet



def main():
    freq = input("Enter Currency Rate Frequency (DAILY, MONTHLY, YEARLY):")
    sym = input("Enter Currency SYMBOL (Ex: BTC):")
    mark = input("Enter Currency Market (USD):")

    DIGITAL_CURRENCY = "DIGITAL_CURRENCY_"+freq+""
    SYMBOL = sym
    MARKET = mark
    API_KEY = "demo"

    market_data = get_past_market_data(DIGITAL_CURRENCY, SYMBOL, MARKET, API_KEY)

    organized_arr = organize_data_for_pandas(market_data, MARKET)

    run_prophet(organized_arr)

if __name__ == "__main__":
    main()

Enter Currency Rate Frequency (DAILY, MONTHLY, YEARLY):DAILY
Enter Currency SYMBOL (Ex: BTC):ETH
Enter Currency Market (USD):USD


KeyError: ignored